In [234]:
import tensorflow as tf
import numpy as np

Imports of all libraries needed for the implementation

In [235]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

mnist_data = input_data.read_data_sets('MNIST_data',  one_hot=True)

#get datasets from mnist
training_data, training_labels = mnist_data.train.images, mnist_data.train.labels
testing_data, testing_labels = mnist_data.test.images, mnist_data.test.labels

#define work variables that will pass the data to the network
data_pl = tf.placeholder(tf.float32, shape=(None, mnist.IMAGE_PIXELS))
labels_pl = tf.placeholder(tf.float32, shape=(None,10))

# data.train.labels.size

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Import and load dataset. <br/>
The MNIST data is split into three parts: 55,000 data points of training data __(mnist.train)__, 10,000 points of test data __(mnist.test)__, and 5,000 points of validation data __(mnist.validation)__. Method that will read and return placeholders with the data and labels from the mnist dataset, stored into tensorflow variables.


In [236]:
mnist_dataset = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


We define input parameters required for this simulation.

In [237]:
reference_tau = tf.constant(10, dtype=tf.float32)
rc_tau = tf.constant(5, dtype=tf.float32)
voltage_threshold = tf.constant(3, dtype=tf.float32)
gamma = 0.2
one = tf.constant(1,dtype=tf.float32)

We define all work variables needed, such as weights for the artificial neural network. Since there is no way to convert the bias variables of an artificial network into parameters of a spiking network, we will omit using them. 

In [238]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

Now that the environment is defined we can specify the neuron model to be used.

In [239]:
def kernel(input_signal):
    data_norm = tf.add(one,tf.exp(tf.truediv(input_signal,gamma)))
    smoothed_data = tf.scalar_mul(gamma, tf.log(data_norm))
    return smoothed_data

In [240]:
def lif_neuron(prev_layer):
    kernel_res = kernel(tf.sub(prev_layer,voltage_threshold))
    rescaled_kernel = tf.log(tf.add(one,tf.truediv(voltage_threshold, kernel_res)))
    neuron_act = tf.truediv(one,(tf.add(reference_tau, tf.scalar_mul(rc_tau, rescaled_kernel)))) 
    return neuron_act

We have defined the dynamics of our neuron model, this will replace the ReLU or softmax activation functions typically used for artificial neurons.<br/>
The model has been derived from the Leaky integrate and fire spiking neuron model steady state firing rate, to which some smoothing has been applied in order to make it suitable for gradient descent learning.

In [241]:
weights_hidden = init_weights([mnist.IMAGE_PIXELS, 625])
weights_output = init_weights([625, 10])

Here we should define all the weight variable for the network, depending on how many layers we want, the weight tensors should probably be linked with their specific layer.

In [242]:
def model(data, weights_h, weights_o):
    hidd_layer = lif_neuron(tf.matmul(data, weights_h))
    out_pred = lif_neuron(tf.matmul(hidd_layer, weights_o))
    return out_pred

We define a function that returns the output layer of the network, so that we can then compute the cost of the misclassifications over the training batch. We use the lif_neuron model to determine the behaviour of the individual neurons. 

In [243]:
neuron_pred = model(data_pl, weights_hidden, weights_output)
# tf.Print(neuron_pred,[neuron_pred], 'the prediction is: ')
error = tf.reduce_mean(tf.square(tf.sub(tf.nn.softmax(neuron_pred), labels_pl)))
train_proc = tf.train.GradientDescentOptimizer(0.2).minimize(error)

predict_proc = tf.argmax(neuron_pred, 1)

Define a session parameter,we can also use an interactive session parameter that would allows us to mix operations that build the computational graph with ones that run the graph, otherwise all the variables and graph parameters would have to be defined before we can run the graph, like in this case. An interactive session can be defined before this point, so it would allow us to still create more variables before it is called.

In [244]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in range(20):
        for start, end in zip(range(0, len(training_data), 128), range(128, len(training_data+1), 128)):
            sess.run(train_proc, feed_dict={data_pl:training_data[start:end], labels_pl:training_labels[start:end]})
#             print(sess.run(predict_proc, feed_dict={data_pl:training_data[start:end], labels_pl:training_labels[start:end]}))
            #tf.Print(labels_pl,[labels_pl], 'the true label is: ')
        print(i, np.mean(np.argmax(testing_labels, axis=1) == 
                         sess.run(predict_proc, feed_dict={data_pl:testing_data, labels_pl:testing_labels})))

(0, 0.098000000000000004)
(1, 0.098000000000000004)
(2, 0.098000000000000004)
(3, 0.098000000000000004)
(4, 0.098000000000000004)
(5, 0.098000000000000004)
(6, 0.098000000000000004)
(7, 0.098000000000000004)
(8, 0.098000000000000004)
(9, 0.098000000000000004)
(10, 0.098000000000000004)
(11, 0.098000000000000004)
(12, 0.098000000000000004)
(13, 0.098000000000000004)
(14, 0.098000000000000004)
(15, 0.098000000000000004)
(16, 0.098000000000000004)
(17, 0.098000000000000004)
(18, 0.098000000000000004)
(19, 0.098000000000000004)
